created on: Tue Jan 14 09:44:36 2020
<br>
Group 7
<br>
@authors: V.B., E.G.

# Environment

# Librairies

In [108]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import h5py
import warnings
warnings.filterwarnings('ignore')
import datetime

# Deep learning
import keras
from keras.models import load_model
from keras.applications.imagenet_utils import decode_predictions

from keras.applications.vgg16 import preprocess_input

In [109]:
time_s = datetime.datetime.now()

In [133]:
project_path = './../'
models_path = project_path + 'Models_F/'
data_path = 'Interpromo2020/All Data/ANALYSE IMAGE/'
path_out = './'

# Choose social network: SEATGURU, INSTAGRAM
social_net = 'SEATGURU'
insta_hashtag = 'boeing'  # if social_net == 'INSTAGRAM'

# Choose images parameters
size = (224, 224)
greys = False

In [ ]:
path_real = project_path + 'ImagesStats/'
df_real = pd.read_csv(path_real + 'g7_SEATGURU_annotate.csv', sep=';')
df_real.rename(columns={"name": "img"}, inplace = True)

In [111]:
##Functions

In [112]:
%run g7_pickle_save_load.ipynb

# Read images to predict

In [113]:
def read_img(img_path: str, size: tuple, greys: bool=False) -> np.array:
    
    img_img = Image.open(img_path)
    img_arr = np.array(img_img.resize(size))
    #img_arr = img_arr.reshape(1, size[0], size[1], 1 if greys else 3)
    img_arr = preprocess_input(img_arr.reshape(1,size[0], size[1], 1 if greys else 3))
    
    return img_arr

In [114]:
# Path
if social_net == 'SEATGURU':
    folder = 'IMG ' + social_net + '/'

elif social_net == 'INSTAGRAM':
    folder = social_net + '/' + insta_hashtag + '/'
    
path_pred = project_path + data_path + folder
imgs_names = os.listdir(path_pred)
imgs_names = [img for img in imgs_names if '.jpg' in img]

# Init results DataFrame
df = pd.DataFrame(columns=['img', 
                           'view', 
                           'view_proba', 
                           'manufacturer', 
                           'manufacturer_proba', 
                           'type', 
                           'type_proba'])
df['img'] = imgs_names

In [115]:
# Read all images, convert to array
all_imgs_arr = np.array([read_img(path_pred + imgs_names[k], size=(size[0], size[1]), greys=greys) 
                         for k in range(len(imgs_names))])

# Reshape for prediction
all_imgs_arr = all_imgs_arr.reshape(len(imgs_names), size[0], size[1], 1 if greys else 3)
all_imgs_arr.shape

(2556, 224, 224, 3)

# Models pipeline

#### Functions

In [116]:
def predict_from_model(img_arr: np.array, models_path: str, model_name: str) -> (list, list):
    
    # Load model and predict
    model, dic_class = load_files_model(path_mod=models_path, mod_name=model_name)
    print(dic_class)
    preds = model.predict(img_arr)
    labels = [np.argmax(preds[k]) for k in range(len(preds))]
    proba_labels = [np.max(preds[k]) for k in range(len(preds))]
    labels = [list(dic_class.keys())[list(dic_class.values()).index(lab)] for lab in labels]
    
    del model, dic_class  # remove model and dict from environment
    
    return labels, proba_labels

In [117]:
def predict_save(df: pd.DataFrame, imgs_arr: np.array, filter_:list, models_path: str, 
                 model_name: str, to_fill: str) -> (pd.DataFrame, int):
    
    # Apply filter(s)
    for k in range(len(filter_)):
        col = list(filter_.keys())[k]
        val = list(filter_.values())[k]
        i = df[df[col] == val].index
        imgs_arr = all_imgs_arr[i]

    # Get labels with probabilities
    labels, proba_labels = predict_from_model(img_arr=imgs_arr, models_path=models_path,
                                              model_name=model_name)

    # Fill DataFrame
    df[to_fill].loc[i] = labels
    df[to_fill + '_proba'].loc[i] = proba_labels
    
    return df, i

## First step: view

In [118]:
labels, proba_labels = predict_from_model(img_arr=all_imgs_arr, models_path=models_path,
                                          model_name='View')

# Fill DataFrame with View labels
df['view'] = labels
df['view_proba'] = proba_labels

{'Ext': 0, 'Ext_Int': 1, 'Int': 2, 'Meal': 3}


In [119]:
df

,img,view,view_proba,manufacturer,manufacturer_proba,type,type_proba
0,Cathay_Pacific_Airways_Boeing_777-300ER_C_0.jpg,Meal,0.999996,NaN,NaN,NaN,NaN
1,KLM_Airbus_A330-300_1.jpg,Int,0.999998,NaN,NaN,NaN,NaN
2,American_Airlines_Boeing_767-300_3.jpg,Int,1.000000,NaN,NaN,NaN,NaN
3,Air_Canada_Boeing_767-300ER_v2_3.jpg,Meal,0.999946,NaN,NaN,NaN,NaN
4,United_Airlines_Q400_A_2.jpg,Ext,0.999990,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2551,Air_Canada_Airbus_A330_C_0.jpg,Int,0.999976,NaN,NaN,NaN,NaN
2552,Qatar_Airways_Airbus_A321_2.jpg,Int,0.999851,NaN,NaN,NaN,NaN
2553,Frontier_Airlines_Airbus_A320_4.jpg,Ext_Int,0.999627,NaN,NaN,NaN,NaN
2554,Spirit_Airlines_Airbus_A320_V2_3.jpg,Int,0.998991,NaN,NaN,NaN,NaN


In [120]:
df.head()

,img,view,view_proba,manufacturer,manufacturer_proba,type,type_proba
0,Cathay_Pacific_Airways_Boeing_777-300ER_C_0.jpg,Meal,0.999996,NaN,NaN,NaN,NaN
1,KLM_Airbus_A330-300_1.jpg,Int,0.999998,NaN,NaN,NaN,NaN
2,American_Airlines_Boeing_767-300_3.jpg,Int,1.000000,NaN,NaN,NaN,NaN
3,Air_Canada_Boeing_767-300ER_v2_3.jpg,Meal,0.999946,NaN,NaN,NaN,NaN
4,United_Airlines_Q400_A_2.jpg,Ext,0.999990,NaN,NaN,NaN,NaN


In [122]:
df_real

,img,format,height,width,height_to_width,ncol,aircraft_manufacturer,aircraft_type,view
0,Cathay_Pacific_Airways_Boeing_777-300ER_C_0.jpg,jpg,540,960,0.5625,3,Boeing,777,Meal
1,KLM_Airbus_A330-300_1.jpg,jpg,720,960,0.7500,3,Airbus,A330,Int
2,American_Airlines_Boeing_767-300_3.jpg,jpg,720,720,1.0000,3,Boeing,767,Int
3,Air_Canada_Boeing_767-300ER_v2_3.jpg,jpg,720,960,0.7500,3,Boeing,767,Meal
4,United_Airlines_Q400_A_2.jpg,jpg,720,960,0.7500,3,Other,NaN,Ext
...,...,...,...,...,...,...,...,...,...
2551,Air_Canada_Airbus_A330_C_0.jpg,jpg,720,960,0.7500,3,Airbus,A330,Int
2552,Qatar_Airways_Airbus_A321_2.jpg,jpg,720,960,0.7500,3,Airbus,A321,Int
2553,Frontier_Airlines_Airbus_A320_4.jpg,jpg,720,720,1.0000,3,Airbus,A320,Ext_Int
2554,Spirit_Airlines_Airbus_A320_V2_3.jpg,jpg,720,960,0.7500,3,Airbus,A320,Int


In [123]:
df_verif = pd.merge(df_real, df, on = "img")
df_verif

,img,format,height,width,height_to_width,ncol,aircraft_manufacturer,aircraft_type,view_x,view_y,view_proba,manufacturer,manufacturer_proba,type,type_proba
0,Cathay_Pacific_Airways_Boeing_777-300ER_C_0.jpg,jpg,540,960,0.5625,3,Boeing,777,Meal,Meal,0.999996,NaN,NaN,NaN,NaN
1,KLM_Airbus_A330-300_1.jpg,jpg,720,960,0.7500,3,Airbus,A330,Int,Int,0.999998,NaN,NaN,NaN,NaN
2,American_Airlines_Boeing_767-300_3.jpg,jpg,720,720,1.0000,3,Boeing,767,Int,Int,1.000000,NaN,NaN,NaN,NaN
3,Air_Canada_Boeing_767-300ER_v2_3.jpg,jpg,720,960,0.7500,3,Boeing,767,Meal,Meal,0.999946,NaN,NaN,NaN,NaN
4,United_Airlines_Q400_A_2.jpg,jpg,720,960,0.7500,3,Other,NaN,Ext,Ext,0.999990,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2551,Air_Canada_Airbus_A330_C_0.jpg,jpg,720,960,0.7500,3,Airbus,A330,Int,Int,0.999976,NaN,NaN,NaN,NaN
2552,Qatar_Airways_Airbus_A321_2.jpg,jpg,720,960,0.7500,3,Airbus,A321,Int,Int,0.999851,NaN,NaN,NaN,NaN
2553,Frontier_Airlines_Airbus_A320_4.jpg,jpg,720,720,1.0000,3,Airbus,A320,Ext_Int,Ext_Int,0.999627,NaN,NaN,NaN,NaN
2554,Spirit_Airlines_Airbus_A320_V2_3.jpg,jpg,720,960,0.7500,3,Airbus,A320,Int,Int,0.998991,NaN,NaN,NaN,NaN


In [124]:
df_verif['pred_view_OK'] = df_verif.apply(lambda x : x['view_x'] == x['view_y'], axis = 1)
len(df_verif[df_verif['pred_view_OK']== True])/len(df)

0.9671361502347418

## Exterior

### Manufacturer and type

In [125]:
# If View == 'Ext', predict aircraft type and fill DataFrame
filter_ = dict({'view': 'Ext'})
df, ind = predict_save(df, all_imgs_arr, filter_, models_path, model_name='Ext_typ', to_fill='type')

# Deduce manufacturer from aircraft type
df['manufacturer'].loc[ind] = ['Airbus' if 'A' in typ else 'Boeing' for typ in df['type'].loc[ind]]

{'737': 0, '747': 1, '757': 2, '777': 3, '787': 4, 'A320': 5, 'A321': 6, 'A330': 7, 'A340': 8, 'A350': 9, 'A380': 10}


In [126]:
df

,img,view,view_proba,manufacturer,manufacturer_proba,type,type_proba
0,Cathay_Pacific_Airways_Boeing_777-300ER_C_0.jpg,Meal,0.999996,NaN,NaN,NaN,NaN
1,KLM_Airbus_A330-300_1.jpg,Int,0.999998,NaN,NaN,NaN,NaN
2,American_Airlines_Boeing_767-300_3.jpg,Int,1.000000,NaN,NaN,NaN,NaN
3,Air_Canada_Boeing_767-300ER_v2_3.jpg,Meal,0.999946,NaN,NaN,NaN,NaN
4,United_Airlines_Q400_A_2.jpg,Ext,0.999990,Boeing,NaN,757,0.44616
...,...,...,...,...,...,...,...
2551,Air_Canada_Airbus_A330_C_0.jpg,Int,0.999976,NaN,NaN,NaN,NaN
2552,Qatar_Airways_Airbus_A321_2.jpg,Int,0.999851,NaN,NaN,NaN,NaN
2553,Frontier_Airlines_Airbus_A320_4.jpg,Ext_Int,0.999627,NaN,NaN,NaN,NaN
2554,Spirit_Airlines_Airbus_A320_V2_3.jpg,Int,0.998991,NaN,NaN,NaN,NaN


In [128]:
df_verif_ext = pd.merge(df_real, df, on = "img")
df_verif_ext = df_verif_ext[df_verif_ext['view_x'] == 'Ext']
df_verif_ext = df_verif_ext[df_verif_ext['aircraft_manufacturer'].isin(['Airbus', 'Boeing'])]
df_verif_ext.head(50)
#df_ext['pred_man_OK'] == 

,img,format,height,width,height_to_width,ncol,aircraft_manufacturer,aircraft_type,view_x,view_y,view_proba,manufacturer,manufacturer_proba,type,type_proba
13,LAN_Airlines_Airbus_A320_4.jpg,jpg,720,960,0.750000,3,Airbus,A320,Ext,Ext,1.000000,Boeing,NaN,787,0.265307
19,Air_Transat_Airbus_A310-300_1.jpg,jpg,640,960,0.666667,3,Airbus,A310,Ext,Ext,1.000000,Airbus,NaN,A380,0.402337
22,KLM_Boeing_737-700_2.jpg,jpg,720,960,0.750000,3,Boeing,737,Ext,Ext,0.999869,Boeing,NaN,777,0.466331
28,Lufthansa_Airbus_A340-300_V3_0.jpg,jpg,720,960,0.750000,3,Airbus,A340,Ext,Ext,1.000000,Boeing,NaN,757,0.27659
36,Turkish_Airlines_Airbus_A340_0.jpg,jpg,720,720,1.000000,3,Airbus,A340,Ext,Ext,0.999991,Airbus,NaN,A320,0.169308
40,LAN_Airlines_Boeing_787-900_0.jpg,jpg,720,540,1.333333,3,Boeing,787,Ext,Ext,0.999919,Boeing,NaN,757,0.192279
48,SpiceJet_Boeing_737-800_new_3.jpg,jpg,720,540,1.333333,3,Boeing,737,Ext,Ext,0.999939,Boeing,NaN,747,0.429489
51,Finnair_Airbus_A321_0.jpg,jpg,720,720,1.000000,3,Airbus,A321,Ext,Ext,1.000000,Airbus,NaN,A330,0.209073
65,Asiana_Boeing_747-400_full_0.jpg,jpg,720,960,0.750000,3,Boeing,747,Ext,Ext,0.999993,Boeing,NaN,747,0.342231
103,Thomas_Cook_Airlines_Airbus_A330-200_V2_0.jpg,jpg,720,960,0.750000,3,Airbus,A330,Ext,Ext,1.000000,Boeing,NaN,757,0.208594


## Interior

### Manufacturer

In [129]:
# If View == 'Int', predict manufacturer
filter_ = dict({'view': 'Int'})
df, ind = predict_save(df, all_imgs_arr, filter_, models_path, model_name='Int_man', to_fill='manufacturer')

{'Airbus': 0, 'Boeing': 1}


### Airbus and Boeing types

In [130]:
# If View == 'Int' and manufacturer == 'Airbus', predict type
filter_ = dict({'view': 'Int',
                'manufacturer': 'Airbus'})
df, ind = predict_save(df, all_imgs_arr, filter_, models_path, model_name='Int_Airbus', to_fill='type')

# If View == 'Int' and manufacturer == 'Boeing', predict type
filter_ = dict({'view': 'Int',
                'manufacturer': 'Boeing'})
df, ind = predict_save(df, all_imgs_arr, filter_, models_path, model_name='Int_Boeing', to_fill='type')

{'A320': 0, 'A321': 1, 'A330': 2, 'A350': 3, 'A380': 4}
{'737': 0, '747': 1, '757': 2, '777': 3}


# Save results as csv

In [134]:
# Path to save
if social_net != 'INSTAGRAM':
    path_save = path_out + '/pred_' + social_net + '_2.csv'
    
else:
    path_save = path_out + '/pred_' + social_net + '_' + insta_hashtag + '_2.csv'

# Save
df.to_csv(path_or_buf=path_save, sep=';', encoding='utf-8', index=False)

In [ ]:
print(datetime.datetime.now() - time_s)